In [1]:
import matplotlib.pyplot as plt
import numpy as np
import dask.array as da
import zarr as zr
import django
from dask.distributed import Client, progress

django.setup()
from elements.models import Zarr
from dask.distributed import Client, progress
c = Client()
import xarray, h5py

Debugging build
Hallo


In [3]:

from metamorphers.models import Metamorphing

In [4]:
met = Metamorphing.objects.last()

In [5]:
array = met.representation.loadArray()

print(array.dtype)
array = array[:, :, :, :, 0]

/code/media/zarr/sample-240
float16


In [6]:
it = array.sel(channels="Ch1-T2")

In [14]:

import ipyvolume as ipv
ipv.quickvolshow(it)

/opt/conda/lib/python3.7/site-packages/ipyvolume/widgets.py:179: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data_view = self.data_original[view]
/opt/conda/lib/python3.7/site-packages/ipyvolume/utils.py:204: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  data = (data[slices1] + data[slices2])/2


In [ ]:
import napari

with napari.gui_qt():
    # specify contrast_limits and is_pyramid=False with big data
    # to avoid unecessary computations
    napari.view_image(it, contrast_limits=[0,2000], is_pyramid=False)

In [7]:
%gui qt

tornado.application - ERROR - Exception in callback functools.partial(<function Kernel.enter_eventloop.<locals>.advance_eventloop at 0x7f26e4fa1950>)
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/tornado/ioloop.py", line 743, in _run_callback
    ret = callback()
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/kernelbase.py", line 314, in advance_eventloop
    eventloop(self)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/eventloops.py", line 129, in loop_qt5
    return loop_qt4(kernel)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel/eventloops.py", line 116, in loop_qt4
    kernel.app = get_app_qt4([" "])
  File "/opt/conda/lib/python3.7/site-packages/IPython/lib/guisupport.py", line 114, in get_app_qt4
    from IPython.external.qt_for_kernel import QtGui
  File "/opt/conda/lib/python3.7/site-packages/IPython/external/qt_for_kernel.py", line 95, in <module>
    QtCore, QtGui, QtSvg, QT_API = load_qt(api_opts)
  File "/opt

In [ ]:
import napari

with napari.gui_qt():
    # specify contrast_limits and is_pyramid=False with big data
    # to avoid unecessary computations
    napari.view_image(it, contrast_limits=[0,2000], is_pyramid=False)

/opt/conda/lib/python3.7/site-packages/qtpy/__init__.py:216: RuntimeWarning: Selected binding "pyqt5" could not be found, using "pyside2"
  'using "{}"'.format(initial_api, API), RuntimeWarning)


In [6]:
cmax = array.max()
cmin = array.min()
high = 255
low = 0

In [7]:
cscale = cmax - cmin
scale = float(high - low) / cscale

In [8]:
scaled = (array - cmin) * scale + low
final = (scaled.clip(low, high) + 0.5)

In [9]:
emptychannel = xarray.DataArray((da.zeros((1256,1256,1,28))), dims=array.dims, coords={"channels":["empty",]})
end = xarray.concat([array,emptychannel],dim="channels")
end.attrs = array.attrs

In [12]:
shape_3d = end.shape[0:3]

In [14]:
rgb_dtype = np.dtype([('R', 'u1'), ('G', 'u1'), ('B', 'u1')])

In [ ]:
contugious = np.ascontiguousarray(end, dtype='u1')